# 使用 LangChain 构建一个 RAG 应用

## RAG 是什么

RAG 是一种将检索到的文档上下文与大语言模型（LLM）结合起来生成答案的技术。

整个过程主要分为以下几个步骤：

1. 加载文档：将原始数据(来源可能是在线网站、本地文件、各类平台等)加载到 LangChain 中。
1. 文档分割：将加载的文档分割成较小的块，以适应模型的上下文窗口，并更容易进行向量嵌入和检索。
1. 存储嵌入：将分割后的文档内容嵌入到向量空间，并存储到向量数据库中，以便后续检索。
1. 检索文档：通过查询向量数据库，检索与问题最相关的文档片段。
1. 生成回答：将检索到的文档片段与用户问题组合，生成并返回答案。

通过这些步骤，可以构建一个强大的问答系统，将复杂任务分解为更小的步骤并生成详细回答。

![rag](../images/rag.png)

In [50]:
! pip install langchain langchain_community langchain_chroma langchainhub

In [5]:
! pip install python-dateutil==2.8.2 typing-extensions==4.9.0

## **RAG 开发指南**

**本指南将详细介绍如何使用 LangChain 框架构建一个基于检索增强生成 (RAG) 的应用。**

下面是基于 LangChain 实现的 RAG 的核心步骤与使用到的关键代码抽象（类型、方法、库等）:

1. **加载文档**: 使用 `WebBaseLoader` 类从指定来源加载内容，并生成 `Document` 对象（依赖 `bs4` 库）。
2. **文档分割**: 使用 `RecursiveCharacterTextSplitter` 类的 `split_documents()` 方法将长文档分割成较小的块。
3. **存储嵌入**: 使用 `Chroma` 类的 `from_documents()` 方法将分割后的文档内容嵌入向量空间，并存储在向量数据库中（使用 `OpenAIEmbeddings`），并可以通过检查存储的向量数量来确认存储成功。。
4. **检索文档**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法基于查询从向量数据库中检索最相关的文档片段。
5. **生成回答**: 使用 `ChatOpenAI` 类的 `invoke()` 方法，将检索到的文档片段与用户问题结合，生成回答（通过 `RunnablePassthrough` 和 `StrOutputParser`）。

我们使用的文档是Lilian Weng撰写的《LLM Powered Autonomous Agents》博客文章（https://lilianweng.github.io/posts/2023-06-23-agent/ ），最终构建好的 RAG 应用支持我们询问关于该文章内容的相关问题。

In [6]:
# 导入必要的库
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

### Step 1: 加载文档

- **描述**: 使用 `DocumentLoader` 从指定来源（如网页）加载内容，并将其转换为 `Document` 对象。
- **重要代码抽象**:
  - 类: `WebBaseLoader`
  - 方法: `load()`
  - 库: `bs4` (BeautifulSoup)
- **代码解释**:
  - **文档加载**: 使用 `WebBaseLoader` 从网页加载内容，并通过 `BeautifulSoup` 解析 HTML，提取重要的部分。
  - **检查加载数量**: 打印加载的文档数量，确保所有文档正确加载。
  - **验证文档内容**: 输出第一个文档的部分内容，确认加载的数据符合预期。

In [13]:
# 使用 WebBaseLoader 从网页加载内容，并仅保留标题、标题头和文章内容
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [14]:
# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度

43131


In [15]:
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])



      LLM Powered Autonomous Agents
    
Date: June 23, 2023  |  Estimated Reading Time: 31 min  |


In [25]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
from langchain.document_loaders.web_base import WebBaseLoader

# Step 1: 获取页面上的文章链接
base_url = "https://coolshell.cn/featured"
response = requests.get(base_url)
if response.status_code == 200:
    # 使用 BeautifulSoup 来解析页面内容，只提取含有文章链接的 <a> 标签
    links_strainer = SoupStrainer("ul", class_="featured-post")
    soup = BeautifulSoup(response.content, "html.parser", parse_only=links_strainer)

    # 获取所有文章的链接，只取前10个
    article_links = [a['href'] for a in soup.find_all('a', href=True)][:10]
else:
    raise Exception(f"Failed to fetch {base_url}, status code: {response.status_code}")

# Step 2: 加载每篇文章的内容
docs = []
for link in article_links:
    # 使用 WebBaseLoader 加载每篇文章的标题、标题头和内容
    loader = WebBaseLoader(
        web_paths=(link,),
        bs_kwargs={"parse_only": SoupStrainer(class_=("entry-title", "entry-content"))},
    )
    docs.extend(loader.load())

# docs 包含了所有文章的内容
# 检查加载的文档内容长度
print(len(docs[0].page_content))  # 打印第一个文档内容的长度
# 查看第一个文档（前100字符）
print(docs[0].page_content[:100])


2307
我看ChatGPT: 为啥谷歌掉了千亿美金
两个月前，我试着想用 ChatGPT 帮我写篇文章《eBPF 介绍》，结果错误百出，导致我又要从头改一遍，从那天我觉得 ChatGPT 生成的内容完全不靠谱


### Step 2: 文档分割

- **描述**: 使用文本分割器将加载的长文档分割成较小的块，以便嵌入和检索。
- **重要代码抽象**:
  - 类: `RecursiveCharacterTextSplitter`
  - 方法: `split_documents()`
- **代码解释**:
  - **文档分割**: 使用 `RecursiveCharacterTextSplitter` 按字符大小分割文档块，设置块大小和重叠字符数，确保文档块适合模型处理。
  - **检查块数量**: 打印分割后的文档块数量，确保分割操作正确执行。
  - **验证块大小**: 输出第一个块的字符数，确认分割块的大小是否符合预期。

In [26]:
# 使用 RecursiveCharacterTextSplitter 将文档分割成块，每块1000字符，重叠200字符
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)

In [27]:
# 检查分割后的块数量和内容
print(len(all_splits))  # 打印分割后的文档块数量

93


In [28]:
print(len(all_splits[0].page_content))  # 打印第一个块的字符数

884


In [31]:
print(all_splits[0].page_content)  # 打印第一个块的内容
print("--------------------------------")
print(all_splits[1].page_content)  # 打印第二个块的内容

我看ChatGPT: 为啥谷歌掉了千亿美金
两个月前，我试着想用 ChatGPT 帮我写篇文章《eBPF 介绍》，结果错误百出，导致我又要从头改一遍，从那天我觉得 ChatGPT 生成的内容完全不靠谱，所以，从那天开始我说我不会再用 ChatGPT 来写文章（这篇文章不是由 ChatGPT 生成），因为，在试过一段时间后，我对 ChatGTP 有基于如下的认识：

ChatGPT 不是基于事实，是基于语言模型的，事实对他来说不重要，对他重要的是他能读懂你的问题，并按照一定的套路回答你的问题。
因为是基于套路的回答，所以，他并不能保证内容是对的，他的目标是找到漂亮的精彩的套路，于是，你会发现，他的内容组织能力和表述还不错，但是只要你认真玩上一段时间，你会发现，ChatGPT 那些表述的套路其实也比较平常一般。它的很多回答其实都不深，只能在表面上。就像 Github 的 Copilot 一样，写不了什么高级的代码，只能帮你写一些常规格式化的代码（当然，这也够了）

ChatGPT 就是一个语言模型，如果不给他足够的数据和信息，它基本就是在胡编乱造
所以，基于上面这两个点认识，以发展的眼光来看问题，我觉得 ChatGPT 这类的 AI 可以成为一个小助理，他的确可以干掉那些初级的脑力工作者，但是，还干不掉专业的人士，这个我估计未来也很难，不过，这也很帅了，因为大量普通的工作的确也很让人费时间和精力，但是有个前提条件——就是ChatGPT所产生的内容必需是真实可靠的，没有这个前提条件的话，那就什么用也没有了。
今天，我想从另外一个角度来谈谈 ChatGPT，尤其是我在Youtube上看完了微软的发布会《Introducing your copilot for the web: AI-powered Bing and Microsoft Edge 》，才真正意识到Google 的市值为什么会掉了1000亿美元，是的，谷歌的搜索引擎的霸主位置受到了前所未有的挑战……

我们先来分析一下搜索引擎解决了什么样的用户问题，在我看来搜索引擎解决了如下的问题：
--------------------------------
我们先来分析一下搜索引擎解决了什么样的用户问题，在我看来搜索引擎解决了如下的问题：

知识或信息索引。查新闻，查股票，查历史，查文档，找答案……
找服务提供商。找卖

In [32]:
print(all_splits[0].metadata)  # 打印第一个块的元数据

{'source': 'https://coolshell.cn/articles/22398.html', 'start_index': 0}


### Step 3: 存储嵌入

- **描述**: 将分割后的文档内容嵌入到向量空间中，并存储到向量数据库，以便后续检索。
- **重要代码抽象**:
  - 类: `Chroma`
  - 方法: `from_documents()`
  - 类: `OpenAIEmbeddings`
- **代码解释**:
  - **存储嵌入**: 使用 `Chroma.from_documents()` 方法将所有分割的文档片段进行嵌入(`OpenAIEmbeddings`嵌入模型)，将文档片段嵌入向量空间，并存储在向量数据库中。

#### Chroma 基础使用

**下面是初始化 Chroma 数据库（仅实例化，未存储向量数据）的常见做法：**

**使用构造函数初始化**: 在本地持久化存储 Chroma 数据库.

```python
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not neccesary
)
```

**使用 Cleint 初始化**: 更方便地访问底层数据库/集合。

```python
import chromadb

persistent_client = chromadb.PersistentClient()
collection = persistent_client.get_or_create_collection("collection_name")
collection.add(ids=["1", "2", "3"], documents=["a", "b", "c"])

vector_store_from_client = Chroma(
    client=persistent_client,
    collection_name="collection_name",
    embedding_function=embeddings,
)
```


**我们直接使用 `Chroma.from_documents()` 方法 实例化+数据存储**:

该方法返回 Chroma 实例，数据类型为`langchain_chroma.vectorstores.Chroma`，详细 API 文档： https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html

In [39]:
# 使用 Chroma 向量存储和 OpenAIEmbeddings 模型，将分割的文档块嵌入并存储
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OpenAIEmbeddings()
)

In [40]:
# 查看 vectorstore 数据类型
type(vectorstore) 

langchain_chroma.vectorstores.Chroma

### Step 4: 检索文档

- **描述**: 使用 `VectorStoreRetriever` 类的 `as_retriever()` 和 `invoke()` 方法，从向量数据库中检索与查询最相关的文档片段。
- **重要代码抽象**:
  - 类: `VectorStoreRetriever`
  - 方法: `as_retriever()`, `invoke()`
- **代码解释**:
  - **文档检索**: 将向量存储转换为检索器，并基于查询执行相似性搜索，获取相关文档片段。
  - **检查检索数量**: 打印检索到的文档片段数量，确保检索操作成功。
  - **验证检索内容**: 输出第一个检索到的文档内容，确认检索结果与预期相符。

在 LangChain 中，所有向量数据库都支持**vectorstore.as_retriever** 方法，实例化该数据库对应的检索器（Retriever），数据类型为`VectorStoreRetriever`，详细 API 文档：https://python.langchain.com/v0.2/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStoreRetriever.html

In [41]:
# 使用 VectorStoreRetriever 从向量存储中检索与查询最相关的文档
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [42]:
type(retriever)

langchain_core.vectorstores.VectorStoreRetriever

In [43]:
retrieved_docs = retriever.invoke("什么是eBPF？")

In [44]:
# 检查检索到的文档内容
print(len(retrieved_docs))  # 打印检索到的文档数量

6


In [47]:
print(retrieved_docs[0].page_content)  # 打印第一个检索到的文档内容

eBPF 介绍
很早前就想写一篇关于eBPF的文章，但是迟迟没有动手，这两天有点时间，所以就来写一篇，这文章主要还是简单的介绍eBPF 是用来干什么的，并通过几个示例来介绍是怎么玩的，这个技术非常非常之强，Linux 操作系统的观测性实在是太强大了，并在 BCC 加持下变得一览无余。这个技术不是一般的运维人员或是系统管理员可以驾驭的，这个还是要有底层系统知识并有一定开发能力的技术人员才能驾驭的了的。我在这篇文章的最后给了个彩蛋。


目录

介绍用途工作原理示例延伸阅读彩蛋
介绍
eBPF（extened Berkeley Packet Filter）是一种内核技术，它允许开发人员在不修改内核代码的情况下运行特定的功能。eBPF 的概念源自于 Berkeley Packet Filter（BPF），后者是由贝尔实验室开发的一种网络过滤器，可以捕获和过滤网络数据包。
出于对更好的 Linux 跟踪工具的需求，eBPF 从 dtrace中汲取灵感，dtrace 是一种主要用于 Solaris 和 BSD 操作系统的动态跟踪工具。与 dtrace 不同，Linux 无法全面了解正在运行的系统，因为它仅限于系统调用、库调用和函数的特定框架。在Berkeley Packet Filter  (BPF)（一种使用内核 VM 编写打包过滤代码的工具）的基础上，一小群工程师开始扩展 BPF 后端以提供与 dtrace 类似的功能集。 eBPF 诞生了。2014 年随 Linux 3.18 首次限量发布，充分利用 eBPF 至少需要 Linux 4.4 以上版本。


### Step 5: 生成回答

- **描述**: 将之前构建的组件（检索器、提示、LLM等）组合成一个完整的链条，实现用户问题的检索与生成回答。完整链条：输入用户问题，检索相关文档，构建提示，将其传递给模型（使用`ChatOpenAI` 类的 `invoke()` 方法），并解析输出生成最终回答。
- **重要代码抽象**:
  - 类: `ChatOpenAI`
  - 方法: `invoke()`
  - 类: `RunnablePassthrough`
  - 类: `StrOutputParser`
  - 模块：`hub`
- **代码解释**:
  - **模型初始化**: 使用 `ChatOpenAI` 类初始化一个 `GPT-4o-mini` 模型，准备处理生成任务。
  - **文档格式化**: 定义 `format_docs` 函数，用于将检索到的文档内容格式化为字符串。
  - **构建 RAG 链**: 使用 LCEL (LangChain Execution Layer) 的 `|` 操作符将各个组件连接成一个链条，包括文档检索、提示构建、模型调用以及输出解析。
  - **生成回答**: 使用 `stream()` 方法逐步输出生成的回答，并实时展示，确保生成的结果符合预期。

![retrieval](../images/retrieval.png)

#### LangChain Hub

`LangChain Hub` (https://smith.langchain.com/hub) 是一个提示词模板开源社区，为开发者提供了大量开箱即用的提示词模板。属于 `LangSmith` 产品的一部分。

下面我们尝试使用 RAG 应用的提示词模板：https://smith.langchain.com/hub/rlm/rag-prompt


```
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
```

In [48]:
# 定义 RAG 链，将用户问题与检索到的文档结合并生成答案
llm = ChatOpenAI(model="gpt-4o-mini")

In [72]:
# 使用 hub 模块拉取 rag 提示词模板
prompt = hub.pull("krunal/more-crafted-rag-prompt")

/Users/chaucer/Code/github/openai-quickstart/.venv/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [80]:
# 打印模板
print(prompt.template)

# Your role
You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer to the questioner's needs from the documents you are given.


# Instruction
Your task is to answer the question using the following pieces of retrieved context delimited by XML tags.

<retrieved context>
Retrieved Context:
{context}
</retrieved context>


# Constraint
1. Think deeply and multiple times about the user's question\nUser's question:\n{question}\nYou must understand the intent of their question and provide the most appropriate answer.
- Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
3. Generate a concise, logical answer. When generating the answer, Do

In [77]:
# 为 context 和 question 填充样例数据，并生成 ChatModel 可用的 Messages
example_messages = prompt.invoke(
    {"context": "filler context", "question": "filler question"}
).to_messages()

In [78]:
# 查看提示词
print(example_messages[0].content)

# Your role
You are a brilliant expert at understanding the intent of the questioner and the crux of the question, and providing the most optimal answer to the questioner's needs from the documents you are given.


# Instruction
Your task is to answer the question using the following pieces of retrieved context delimited by XML tags.

<retrieved context>
Retrieved Context:
filler context
</retrieved context>


# Constraint
1. Think deeply and multiple times about the user's question\nUser's question:\nfiller question\nYou must understand the intent of their question and provide the most appropriate answer.
- Ask yourself why to understand the context of the question and why the questioner asked it, reflect on it, and provide an appropriate response based on what you understand.
2. Choose the most relevant content(the key content that directly relates to the question) from the retrieved context and use it to generate an answer.
3. Generate a concise, logical answer. When generating the 

#### ⭐️**LCEL 在 RAG 中的应用**⭐️

##### **LCEL 概述**

LCEL 是 LangChain 中的一个重要概念，它提供了一种统一的接口，允许不同的组件（如 `retriever`, `prompt`, `llm` 等）可以通过统一的 `Runnable` 接口连接起来。每个 `Runnable` 组件都实现了相同的方法，如 `.invoke()`、`.stream()` 或 `.batch()`，这使得它们可以通过 `|` 操作符轻松连接。

##### **LCEL 中处理的组件**

- **Retriever**: 负责根据用户问题检索相关文档。
- **Prompt**: 根据检索到的文档构建提示，供模型生成回答。
- **LLM**: 接收提示并生成最终的回答。
- **StrOutputParser**: 解析 LLM 的输出，只提取字符串内容，供最终显示。

##### **LCEL 运作机制**

- **构建链条**: 通过 `|` 操作符，我们可以将多个 `Runnable` 组件连接成一个 `RunnableSequence`。LangChain 会自动将一些对象转换为 `Runnable`，如将 `format_docs` 转换为 `RunnableLambda`，将包含 `"context"` 和 `"question"` 键的字典转换为 `RunnableParallel`。

- **数据流动**: 用户输入的问题会在 `RunnableSequence` 中依次经过各个 `Runnable` 组件。首先，问题会通过 `retriever` 检索相关文档，然后通过 `format_docs` 将这些文档转换为字符串。`RunnablePassthrough` 则直接传递原始问题。最后，这些数据被传递给 `prompt` 来生成完整的提示，供 LLM 使用。

##### **LCEL 中的关键操作**

- **格式化文档**: `retriever | format_docs` 将问题传递给 `retriever` 生成文档对象，然后通过 `format_docs` 将这些文档格式化为字符串。
- **传递问题**: `RunnablePassthrough()` 直接传递原始问题，保持原样。
- **构建提示**: `{"context": retriever | format_docs, "question": RunnablePassthrough()} | prompt` 构建完整的提示。
- **运行模型**: `prompt | llm | StrOutputParser()` 运行 LLM 生成回答，并解析输出。

#### 使用 LCEL 构建 RAG Chain

下面我们将 LCEL 的概念与代码实现结合起来，展示了如何通过一系列 `Runnable` 组件来实现完整的 RAG 流程。通过 LCEL，LangChain 提供了高度模块化和可扩展的开发方式，使复杂任务的实现变得更加简单和高效。


In [81]:
# 定义格式化文档的函数
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [82]:
# 使用 LCEL 构建 RAG Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [83]:
# 流式生成回答
for chunk in rag_chain.stream("如何做团队协同？国内有哪些团队协同软件？"):
    print(chunk, end="", flush=True)

团队协同的关键在于建立信任的企业文化和科学的管理思维，这样才能有效使用协同工具。国内常用的团队协同软件包括企业微信、钉钉和飞书，它们主要通过群组的方式进行沟通，而这与国外的Slack和Discord等工具有所不同，后者更注重长期的频道管理。为了实现高效的协同，建议使用创作类工具，如Google Docs、Github和Figma，这些工具不仅支持实时协作，还能提升信息的结构化和准确性。最终，好的协同工具应能促进团队成员之间的互动与激励，提升整体工作效率。

In [84]:
# 流式生成回答
for chunk in rag_chain.stream("讲一讲TCP 的 TIME_WAIT 的问题"):
    print(chunk, end="", flush=True)

TCP的TIME_WAIT状态是TCP协议中一个重要的阶段，它主要用于确保数据的可靠传输。在TCP连接关闭后，处于TIME_WAIT状态的连接会等待一个时间段，通常是2倍的最大报文生存时间（MSL），以确保所有的分组都能够从网络中消失，防止旧的重复分组影响新的连接。这个状态不仅帮助确认远程端已经接收到最后的ACK包，还能避免在相同的端口号上建立新连接时出现混淆。然而，过多的TIME_WAIT状态可能导致系统资源的浪费，从而影响新的TCP连接的建立。为此，可以通过调整系统参数如tcp_tw_reuse来重用TIME_WAIT状态，从而缓解资源消耗问题，但这并不能根本解决TIME_WAIT过多的现象。

In [85]:
# 流式生成回答
for chunk in rag_chain.stream("什么是eBPF？"):
    print(chunk, end="", flush=True)

eBPF（扩展的伯克利数据包过滤器）是一种强大的内核技术，允许开发人员在不修改内核代码的情况下执行特定功能。它起源于传统的BPF，最初用于网络过滤，随着时间的发展，eBPF被扩展以支持网络监控、安全过滤、性能分析等多种应用场景。eBPF程序通过用户态应用加载并在内核中执行，内核会在特定事件（如系统调用或网络事件）发生时调用这些程序。该技术的灵活性和安全性使其成为高级用户和开发人员的理想选择，尽管它的学习曲线相对较高。总之，eBPF是现代Linux系统中用于增强可观察性和性能分析的重要工具。

# Homework
1. 使用其他的线上文档或离线文件，重新构建向量数据库，尝试提出3个相关问题，测试 LCEL 构建的 RAG Chain 是否能成功召回。
2. 重新设计或在 LangChain Hub 上找一个可用的 RAG 提示词模板，测试对比两者的召回率和生成质量。

### 自定义 Prompt 的示例

In [27]:
from langchain_core.prompts import PromptTemplate

# 自定义提示词模板
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""

custom_rag_prompt = PromptTemplate.from_template(template)

In [28]:
# 为 context 和 question 填充样例数据，生成 LLM 可用的提示词
print(custom_rag_prompt.invoke({"context": "filler context", "question": "filler question"}).text)

Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

filler context

Question: filler question

Helpful Answer:


In [29]:
# 重新自定义 RAG Chain
custom_rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

In [30]:
# 使用自定义 prompt 生成回答
custom_rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller and more manageable steps, allowing for easier execution and understanding. Techniques such as Chain of Thought (CoT) help enhance model performance by guiding the model to reason step by step. Additionally, the Tree of Thoughts method explores multiple reasoning possibilities at each step, creating a structured approach to problem-solving. Thanks for asking!'